In [1]:
!pip install --upgrade tables
!pip install eli5
!pip install xgboost
!pip install hyperopt

     |████████████████████████████████| 4.3MB 4.9MB/s 
  Found existing installation: tables 3.4.4
    Uninstalling tables-3.4.4:
      Successfully uninstalled tables-3.4.4
     |████████████████████████████████| 112kB 4.7MB/s 


In [2]:
import pandas as pd
import numpy as np

# from sklearn.dummy import DummyRegressor
# from sklearn.tree import DecisionTreeRegressor
# from sklearn.ensemble import RandomForestRegressor

import xgboost as xgb

from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import cross_val_score, KFold

from hyperopt import hp, fmin, tpe, STATUS_OK

import eli5
from eli5.sklearn import PermutationImportance

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)
Using TensorFlow backend.


In [3]:
cd /content/drive/My Drive/Colab Notebooks/Matrix2/data

/content/drive/My Drive/Colab Notebooks/Matrix2/data


In [4]:
df = pd.read_hdf('/content/drive/My Drive/Colab Notebooks/Matrix2/data/car.h5')
print (df.shape)
df.sample(3)

(106494, 155)


,breadcrumb,created_at,price_currency,price_details,price_value,seller_address,seller_name,seller_type,feature_czujniki-parkowania-przednie,feature_poduszka-powietrzna-chroniąca-kolana,feature_kurtyny-powietrzne,feature_klimatyzacja-dwustrefowa,feature_światła-led,feature_czujnik-zmierzchu,feature_elektrycznie-ustawiane-lusterka,feature_asr-(kontrola-trakcji),feature_poduszka-powietrzna-kierowcy,feature_cd,feature_elektryczne-szyby-przednie,feature_poduszka-powietrzna-pasażera,feature_system-start-stop,feature_światła-do-jazdy-dziennej,feature_komputer-pokładowy,feature_elektryczne-szyby-tylne,feature_klimatyzacja-manualna,feature_tapicerka-welurowa,feature_czujnik-deszczu,feature_światła-przeciwmgielne,feature_ogrzewanie-postojowe,feature_radio-niefabryczne,feature_regulowane-zawieszenie,feature_ogranicznik-prędkości,feature_zmieniarka-cd,feature_szyberdach,feature_isofix,feature_asystent-pasa-ruchu,feature_alufelgi,feature_bluetooth,feature_nawigacja-gps,feature_asystent-parkowania,...,param_particle-filter,param_zarejestrowany-w-polsce,param_oferta-od,param_moc,param_make,param_kraj-pochodzenia,param_engine-power,param_year,param_serwisowany-w-aso,param_rodzaj-paliwa,param_możliwość-finansowania,param_registered-in-poland,param_transmission,param_miesięczna-rata,param_kod-silnika,param_version,param_opłata-początkowa,param_model-pojazdu,param_pojemność-skokowa,param_kierownica-po-prawej-(anglik),param_fuel-type,param_vat-free,param_acrylic,param_damaged,param_door-count,param_tuning,param_filtr-cząstek-stałych,param_nr-of-seats,param_kolor,param_engine-capacity,param_pearl,param_stan,param_wersja,param_emisja-co2,param_body-type,param_matowy,param_bezwypadkowy,param_akryl-(niemetalizowany),param_monthly-payment-value,car_id
156218,"[Osobowe, Volkswagen, Transporter, Volkswagen ...","10:53, 27 lutego 2018",PLN,Cena Brutto,7590.0,"Wejherowska 37 - 84-240 Reda, wejherowski, Pom...",Auto Komis Dakar,Dealer,False,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,...,None,Tak,Firmy,102 KM,None,None,None,None,None,Diesel,None,None,None,None,None,None,None,Transporter,2 461 cm3,None,None,None,None,None,None,None,None,None,Niebieski,None,None,Używane,T4,None,None,None,None,None,None,156218
112417,"[Osobowe, Mercedes-Benz, Klasa A, A 170, Merce...","13:12, 2 marca 2018",PLN,"Cena Brutto, Do negocjacji",5600.0,"Pocztowa 1 - 64-700 Czarnków, czarnkowsko-trzc...",AUTO-HANDEL RACZEK Marcin Raczewski,Dealer,False,False,False,False,False,False,True,True,True,True,True,True,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,...,None,None,Firmy,90 KM,None,None,None,None,Tak,Diesel,None,None,None,None,A 170,None,None,Klasa A,1 700 cm3,None,None,None,None,None,None,None,None,None,Szary,None,None,Używane,W168 (1997-2004),None,None,None,Tak,None,None,112417
81040,"[Osobowe, BMW, Seria 5, 520, BMW Seria 5 G30 (...","10:04, 5 marca 2018",PLN,Cena Brutto,244200.0,"Rokicińska 190 - 92-412 Łódź, Widzew (Polska)",Premium Arena,Dealer,True,False,True,False,True,True,True,True,True,True,True,True,True,True,True,True,False,False,True,True,False,False,True,False,False,False,True,False,True,True,True,True,...,None,None,Firmy,190 KM,None,None,None,None,None,Diesel,None,None,None,None,520,None,None,Seria 5,2 000 cm3,None,None,None,None,None,None,None,Tak,None,Szary,None,None,Nowe,G30 (2017-),None,None,None,Tak,None,None,81040


In [5]:
SUFFIX_CAT = '__cat'
for feat in df.columns:
  if isinstance(df[feat][0], list): continue

  factorized_values = df[feat].factorize()[0]
  
  if SUFFIX_CAT in feat:
    df[feat] = factorized_values
  else:
    df[feat + SUFFIX_CAT] = factorized_values

cat_feats = [x for x in df.columns if SUFFIX_CAT in x]
cat_feats = [x for x in cat_feats if 'price' not in x]
print (len(cat_feats))

151


In [0]:
df['param_rok-produkcji'] = df['param_rok-produkcji']. map(lambda x: -1 if str(x) == 'None' else int(x))
df['param_moc'] = df['param_moc']. map(lambda x: -1 if str(x) == 'None' else int(x.split(' ')[0])) #    ||-1 if str(x) == 'None' else int(x))
df['param_pojemność-skokowa'] = df['param_pojemność-skokowa'].map(lambda x: -1 if str(x) == 'None' else int(x.split('cm')[0].replace(' ','')))

In [9]:
feats = [
  'feature_kamera-cofania__cat', 
  'feature_łopatki-zmiany-biegów__cat', 
  'param_napęd__cat', 
  'param_skrzynia-biegów__cat', 
  'param_faktura-vat__cat', 
  'feature_asystent-pasa-ruchu__cat', 
  'param_stan__cat', 
  'feature_światła-led__cat', 
  'feature_bluetooth__cat',
  'feature_regulowane-zawieszenie__cat', 
  'feature_wspomaganie-kierownicy__cat', 
  'feature_system-start-stop__cat', 
  'feature_światła-do-jazdy-dziennej__cat', 
  'feature_światła-xenonowe__cat', 
  'feature_czujniki-parkowania-przednie__cat', 
  'param_moc', 
  'param_rok-produkcji', 
  'param_pojemność-skokowa', 
  'feature_asystent-parkowania__cat', 
  'seller_name__cat']


xgb_params = {
    'max_depth': 5,
    'n_estimators':50,
    'learning_rate': 0.1, 
    'seed': 0
}

run_model(xgb.XGBRegressor(**xgb_params), feats )

[04:57:31] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[04:57:34] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[04:57:38] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-10418.18729276653, 72.80569459652894)

## HYPEROPT

In [16]:
def obj_func (params):
  print("trading with params: ")
  print(params)

  mean_mae, score_std = run_model(xgb.XGBRegressor(**xgb_params), feats )

  return {'loss': np.abs(mean_mae), 'status': STATUS_OK}


  # space





xgb_reg_params = {
      'learning_rate':      hp.choice('learning_rate', np.arange(0.05, 0.31, 0.05)),
      'max_depth':          hp.choice('max_depth',     np.arange(5, 16, 1, dtype=int)),
      'subsample':          hp.quniform('subsample', 0.5,1,0.05),
      'colsample_bytree':   hp.quniform('colsample_bytree', 0.5,1,0.05),
      'objective':          'reg:squarederror',
      'n_estimators':       100,
      'seed':               0,
  }  # xgb.XGBRegressor(learning_rate=, max_depth=, subsample=,colsample_bytree=,objective=, n_estimators=, seed=)

# RUN

best = fmin(obj_func, xgb_reg_params, algo=tpe.suggest, max_evals=25)

best


trading with params: 
{'colsample_bytree': 1.0, 'learning_rate': 0.25, 'max_depth': 13, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.6000000000000001}
[05:49:47] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[05:49:51] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[05:49:54] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
trading with params: 
{'colsample_bytree': 0.75, 'learning_rate': 0.3, 'max_depth': 14, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.55}
[05:49:58] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[05:50:02] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[05:50:

{'colsample_bytree': 1.0,
 'learning_rate': 4,
 'max_depth': 8,
 'subsample': 0.6000000000000001}